<a href="https://colab.research.google.com/github/moriahsantiago/UPenn-Data-Science-Digital-Learning/blob/main/Week_14_Neural_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Neural Networks and Deep Learning
In this assignment, you will be training and evaluating neural networks and other models for the task of developing automated sensor-free detectors of student affect (i.e. concentration, boredom, confusion, and frustration).

## Dataset

This assignment will utilize student data collected from the ASSISTments digital learning platform. The dataset contains 48,716 rows of data from 838 unique students. Each row represents one "clip" of student activity; a clip is meant to represent approximately 20 seconds of actions, but may vary. Each clip is described by 92 features; these expert-engineered features are the result of taking the average, sum, minimum, and maximum value of 23 action-level features when aggregating to the clip-level (i.e. 23 features x 4 aggregations = 92). The dataset also contains 3,109 observations of affect collected by human coders using the [BROMP protocol](https://d1wqtxts1xzle7.cloudfront.net/36773439/BROMP_2.0_Final-libre.pdf?1424899724=&response-content-disposition=inline%3B+filename%3DBaker_Rodrigo_Ocumpaugh_Monitoring_Proto.pdf&Expires=1708897741&Signature=MkENDA~A6ZDfWOD--VrdUT73ngf4~bQJ48Nq1DOnyZkq~h8zwcSben4URR8MnGipxbgbzxkpRE4pfIaLSBRBq5G62-C3DYdw60Kjx0qTsBCQoIWu6XmqPz6ACzyslcJwc~LA7vDIiJ3MVs1CGVccZnDFaFP6YzAnkAbK3HuZ1UgkT3OsxorsD7p7pbgF0P0WEb6X9NevtNAxNbEbzSN7r0mrjAoESZdFkat~q1eVyAcPhQ-ONGB-aK-FzDImMlC7gjxRqiq2J7Husp5RVFVuQ3v7v-gfvyq7rC8Clabci1EkaaCjbF9qxLiibwl5End3Tre6MQkgV4tu7tY~kSOciQ__&Key-Pair-Id=APKAJLOHF5GGSLRBV4ZA); students were labeled as exhibiting either concentration, boredom, confusion, or frustration. These labels were collected in a round-robin fashion, such that not every row in the dataset contains a label, resulting in a large number of unlabeled data rows.

**The dataset can be downloaded from this direct link:
[ASSISTments Affect Labels and Features](https://drive.google.com/file/d/19v4vzxsvHM_zm2a_9Zvquhf_Z4HGtu2a/view?usp=sharing)**

Versions of this dataset have been used in prior works:

* [Ocumpaugh, J., Baker, R., Gowda, S., Heffernan, N., & Heffernan, C. (2014). Population validity for educational data mining models: A case study in affect detection. *British Journal of Educational Technology*, 45(3), 487-501.](https://bera-journals.onlinelibrary.wiley.com/doi/full/10.1111/bjet.12156)

* [Botelho, A. F., Baker, R. S., & Heffernan, N. T. (2017, June). Improving Sensor-Free Affect Detection Using Deep Learning. *In Proceedings of the 2017 International Conference on Artificial Intelligence in Education*, 40-51. Springer, Cham.](https://link.springer.com/chapter/10.1007/978-3-319-61425-0_4)

#Data Loading and Preprocessing
Download the **student_affect_with_clip_features_and_folds.csv** file from the link above. Run the first code cell below to upload the dataset. The second code cell below uses the pandas library to read the file into a Dataframe and displays the number of rows and columns as well as a sample of the loaded data.

*Note: The dataset has already been folded at the student-level. We will be using the "fold" column of this dataset to apply cross-validation*



In [1]:
from google.colab import files
dataset = files.upload()
filename = list(dataset.keys())[0]
print(f"{filename} has been uploaded")

Saving student_affect_with_clip_features_and_folds.csv to student_affect_with_clip_features_and_folds.csv
student_affect_with_clip_features_and_folds.csv has been uploaded


In [2]:
import numpy as np
import pandas as pd
import pickle as pk
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Define the prefixes
prefixes = ["avg_", "sum_", "min_", "max_"]

TARGET_FEATURES = ['confusion', 'concentration', 'frustration', 'boredom']

data = pd.read_csv(filename)
EXPERT_FEATURES = [col for col in data.columns if any(col.lower().startswith(prefix.lower()) for prefix in prefixes)]
data[TARGET_FEATURES] = data[TARGET_FEATURES].fillna(0)

# Print the shape of the dataset
print("\nShape of the dataset (rows, columns):", data.shape)

data


Shape of the dataset (rows, columns): (48716, 106)


,row_id,clip_id,skill,problem_id,user_id,assignment_id,assistment_id,avg_attemptCount,avg_bottomHint,avg_correct,...,sum_totalFrPercentPastWrong,sum_totalFrSkillOpportunities,sum_totalFrTimeOnSkill,confusion,concentration,boredom,frustration,urbanicity,clip_sequence,fold
0,0,1,281,136.00000,72720,287761.0,136.0,1.0,0.0,0.000000,...,0.000000,0,0.00000,0.0,0.0,0.0,0.0,1,1,1
1,1,2,281,136.00000,72720,287761.0,136.0,2.0,0.0,0.000000,...,0.000000,1,186.65000,0.0,0.0,0.0,0.0,1,1,1
2,2,3,24,4468.00000,72720,287767.0,4468.0,1.5,0.0,0.000000,...,0.000000,1,54.56500,0.0,0.0,0.0,0.0,1,1,1
3,3,5,24,4464.00000,72720,287767.0,4468.0,1.5,0.0,0.500000,...,0.500000,5,133.86500,0.0,0.0,0.0,0.0,1,1,1
4,4,7,42,4465.00000,72720,287767.0,4468.0,3.5,0.0,0.166667,...,0.000000,15,836.40199,0.0,0.0,0.0,0.0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48711,48720,92477,78,86278.50000,155781,529087.0,47799.0,1.0,0.0,1.000000,...,0.500000,14,213.01198,0.0,0.0,0.0,0.0,3,838,4
48712,48721,92479,79,87298.50000,155781,529088.0,48669.5,1.0,0.0,1.000000,...,0.904167,31,458.56597,0.0,0.0,0.0,0.0,3,838,4
48713,48722,92481,47,84974.00000,155781,529090.0,46686.0,1.0,0.0,1.000000,...,0.111111,9,343.83000,0.0,0.0,0.0,0.0,3,838,4
48714,48723,92482,47,84952.00000,155781,529090.0,46664.0,1.5,0.0,0.500000,...,0.100000,21,732.08400,0.0,0.0,0.0,0.0,3,838,4


In [3]:
# Check if we have the right features
print("Number of columns:", len(data.columns))
print("Target features present:", [col for col in TARGET_FEATURES if col in data.columns])
print("Sample of behavioral features:", [col for col in data.columns if any(col.startswith(prefix) for prefix in prefixes)][:5])

Number of columns: 106
Target features present: ['confusion', 'concentration', 'frustration', 'boredom']
Sample of behavioral features: ['avg_attemptCount', 'avg_bottomHint', 'avg_correct', 'avg_frIsHelpRequest', 'avg_frPast5HelpRequest']


In [4]:
# Properly filter to only behavioral features and targets
EXPERT_FEATURES = [col for col in data.columns if any(col.lower().startswith(prefix.lower()) for prefix in prefixes)]
data_filtered = data[EXPERT_FEATURES + TARGET_FEATURES]

# Print the new shape
print("Filtered dataset shape:", data_filtered.shape)
print("Features kept:", len(EXPERT_FEATURES), "behavioral features +", len(TARGET_FEATURES), "target features")

# Update our main dataframe
data = data_filtered
print("Final dataset shape:", data.shape)

Filtered dataset shape: (48716, 96)
Features kept: 92 behavioral features + 4 target features
Final dataset shape: (48716, 96)


# Defining Utility Functions
The code cell below provides an implementation of AUC for multi-class prediction (following the method suggested by Hand & Till, 2001) as well as for traditional binary prediction tasks.

[Hand, D. J., & Till, R. J. (2001). A simple generalisation of the area under the ROC curve for multiple class classification problems. *Machine learning, 45, 171-186.](https://link.springer.com/article/10.1023/A:1010920819831)

In [5]:
def alen(x):
    return 1 if np.isscalar(x) else len(x)

def auc(actual, predicted, average_over_labels=True, partition=1024.):
    assert len(actual) == len(predicted)

    ac = np.array(actual, dtype=np.float32).reshape((len(actual),-1))
    pr = np.array(predicted, dtype=np.float32).reshape((len(predicted),-1))

    na = np.argwhere([not np.any(np.isnan(i)) for i in ac]).ravel()

    ac = ac[na]
    pr = pr[na]

    label_auc = []
    for i in range(ac.shape[-1]):
        a = np.array(ac[:,i])
        p = np.array(pr[:,i])

        val = np.unique(a)
        if len(val) == 1:
            label_auc.append(np.nan)
            continue

        pos = np.argwhere(a[:] >= np.median(val))
        neg = np.argwhere(a[:] < np.median(val))

        p_div = int(np.ceil(len(pos)/partition))
        n_div = int(np.ceil(len(neg)/partition))

        div = 0
        for j in range(int(p_div)):
            p_range = list(range(int(j * partition), int(np.minimum(int((j + 1) * partition), len(pos)))))
            for k in range(n_div):
                n_range = list(range(int(k * partition), int(np.minimum(int((k + 1) * partition), len(neg)))))


                eq = np.ones((alen(neg[n_range]), alen(pos[p_range]))) * p[pos[p_range]].T == np.ones(
                    (alen(neg[n_range]), alen(pos[p_range]))) * p[neg[n_range]]

                geq = np.array(np.ones((alen(neg[n_range]), alen(pos[p_range]))) *
                               p[pos[p_range]].T >= np.ones((alen(neg[n_range]),
                                                             alen(pos[p_range]))) * p[neg[n_range]],
                               dtype=np.float32)
                geq[eq[:, :] == True] = 0.5
                div += np.sum(geq)

        label_auc.append(div / (alen(pos)*alen(neg)))

    if average_over_labels:
        return np.nanmean(label_auc)
    else:
        return label_auc

## Part 1: Feed Forward Neural Network

The code cell below formats the data for a non-recurrent model (such as a Feed Forward Neural Network or any of the prediction models that have previously been introduced).

The second code cell applies a 5-fold cross-validation on a Feed Forward Neural Network.

**Please follow the instructions in the ASSISTments assignment for modifying and running the cross-validation code cell.**

In [8]:
# Use corrected data from the diagnostic step
keepers = original_data[TARGET_FEATURES].sum(axis=1) == 1
X_nonrecurrent = np.array(original_data[EXPERT_FEATURES][keepers])
y_nonrecurrent = np.array(original_data[TARGET_FEATURES][keepers])
fold_nonrecurrent = np.array(original_data['fold'][keepers])

print("X_nonrecurrent.shape:", X_nonrecurrent.shape)
print("y_nonrecurrent.shape:", y_nonrecurrent.shape)

X_nonrecurrent.shape: (3109, 92)
y_nonrecurrent.shape: (3109, 4)


In [7]:
# Check what columns we currently have
print("Current columns in data:", list(data.columns))
print("Length of current data:", len(data))

# Make sure we have the fold column
# Re-read the original data to get the fold column back
import pandas as pd

# Read the original CSV file again to get all columns including 'fold'
original_data = pd.read_csv(filename)
print("Original data shape:", original_data.shape)
print("Original columns include 'fold':", 'fold' in original_data.columns)

# Now let's properly filter while keeping the fold column
keepers = original_data[TARGET_FEATURES].sum(axis=1) == 1
X_nonrecurrent = np.array(original_data[EXPERT_FEATURES][keepers])
y_nonrecurrent = np.array(original_data[TARGET_FEATURES][keepers])
fold_nonrecurrent = np.array(original_data['fold'][keepers])

print("Filtered data shapes:")
print("X_nonrecurrent.shape:", X_nonrecurrent.shape)
print("y_nonrecurrent.shape:", y_nonrecurrent.shape)
print("fold_nonrecurrent.shape:", fold_nonrecurrent.shape)

Current columns in data: ['avg_attemptCount', 'avg_bottomHint', 'avg_correct', 'avg_frIsHelpRequest', 'avg_frPast5HelpRequest', 'avg_frPast5WrongCount', 'avg_frPast8HelpRequest', 'avg_frPast8WrongCount', 'avg_frWorkingInSchool', 'avg_hint', 'avg_hintCount', 'avg_hintTotal', 'avg_original', 'avg_past8BottomOut', 'avg_scaffold', 'avg_stlHintUsed', 'avg_timeSinceSkill', 'avg_timeTaken', 'avg_totalFrAttempted', 'avg_totalFrPastWrongCount', 'avg_totalFrPercentPastWrong', 'avg_totalFrSkillOpportunities', 'avg_totalFrTimeOnSkill', 'max_attemptCount', 'max_bottomHint', 'max_correct', 'max_frIsHelpRequest', 'max_frPast5HelpRequest', 'max_frPast5WrongCount', 'max_frPast8HelpRequest', 'max_frPast8WrongCount', 'max_frWorkingInSchool', 'max_hint', 'max_hintCount', 'max_hintTotal', 'max_original', 'max_past8BottomOut', 'max_scaffold', 'max_stlHintUsed', 'max_timeSinceSkill', 'max_timeTaken', 'max_totalFrAttempted', 'max_totalFrPastWrongCount', 'max_totalFrPercentPastWrong', 'max_totalFrSkillOpportun

In [9]:
import keras
import tensorflow as tf
import random
import numpy as np
from keras.models import Sequential
from keras.layers import Masking, Dense, LSTM, TimeDistributed, Dropout, Normalization
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, cohen_kappa_score

# Set a seed value
seed_value= 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

auc_scores = []
kappa_scores = []

for fold in np.unique(fold_nonrecurrent):
    training = np.argwhere(fold_nonrecurrent != fold).ravel()
    testing = np.argwhere(fold_nonrecurrent == fold).ravel()

    X_train, X_test = X_nonrecurrent[training], X_nonrecurrent[testing]
    y_train, y_test = y_nonrecurrent[training], y_nonrecurrent[testing]

    # Define the model
    keras.backend.clear_session()
    model = Sequential([
        Dense(128, activation='relu', input_shape=(92,)), # this represents the input layer and first hidden layer
        Dense(64, activation='relu'),
        Dense(4, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Define early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model with the new training and validation sets
    history = model.fit(X_train, y_train,
                        epochs=100,
                        validation_split=0.2,
                        verbose=1, # setting this to 0 reduces the amount printed
                        callbacks=[early_stopping])

     # Evaluate the model
    y_pred = model.predict(X_test)

    # AUC
    auc_score = auc(y_test, y_pred)

    # Kappa Score
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)
    kappa_score = cohen_kappa_score(y_true_classes, y_pred_classes)

    auc_scores.append(auc_score)
    kappa_scores.append(kappa_score)

# Calculate the average AUC and Kappa scores across all folds
average_auc = np.mean(auc_scores)
average_kappa = np.mean(kappa_scores)

print(f"Average AUC: {average_auc}")
print(f"Average Kappa: {average_kappa}")


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.6948 - loss: 21.6678 - val_accuracy: 0.7480 - val_loss: 4.6558
Epoch 2/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.7351 - loss: 3.3082 - val_accuracy: 0.7077 - val_loss: 3.1212
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7470 - loss: 2.5523 - val_accuracy: 0.7782 - val_loss: 5.2833
Epoch 4/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7513 - loss: 2.9749 - val_accuracy: 0.7097 - val_loss: 4.7769
Epoch 5/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7426 - loss: 2.9634 - val_accuracy: 0.7177 - val_loss: 4.8484
Epoch 6/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7675 - loss: 2.5293 - val_accuracy: 0.7379 - val_loss: 2.6644
Epoch 7/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7692 - loss: 1.5086 - val_accuracy: 0.7823 - val_loss: 2.7646
Epoch 8/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7527 - loss: 2.0817 - val_accuracy: 0.7601 - val_loss: 2.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7067 - loss: 19.3639 - val_accuracy: 0.7306 - val_loss: 4.6841
Epoch 2/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7570 - loss: 3.3349 - val_accuracy: 0.7653 - val_loss: 7.3076
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7827 - loss: 3.3795 - val_accuracy: 0.8143 - val_loss: 4.7459
Epoch 4/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7833 - loss: 4.0677 - val_accuracy: 0.6122 - val_loss: 4.5175
Epoch 5/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7612 - loss: 3.8387 - val_accuracy: 0.8224 - val_loss: 3.4558
Epoch 6/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7909 - loss: 2.6644 - val_accuracy: 0.7735 - val_loss: 2.9220
Epoch 7/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7959 - loss: 2.5058 - val_accuracy: 0.7714 - val_loss: 2.6258
Epoch 8/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7952 - loss: 1.1603 - val_accuracy: 0.7612 - val_loss: 3.

...

64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.5606 - loss: 51.7383 - val_accuracy: 0.5562 - val_loss: 8.3846
Epoch 2/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7152 - loss: 4.9771 - val_accuracy: 0.7495 - val_loss: 3.3885
Epoch 3/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7398 - loss: 3.5556 - val_accuracy: 0.5010 - val_loss: 4.4470
Epoch 4/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7358 - loss: 3.4112 - val_accuracy: 0.6410 - val_loss: 4.0503
Epoch 5/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7485 - loss: 2.9676 - val_accuracy: 0.5740 - val_loss: 3.7962
Epoch 6/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7425 - loss: 2.5045 - val_accuracy: 0.7002 - val_loss: 3.7322
Epoch 7/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7389 - loss: 2.6337 - val_accuracy: 0.5878 - val_loss: 5.9126
Epoch 8/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7474 - loss: 2.8312 - val_accuracy: 0.5878 - val_loss: 3.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6352 - loss: 40.4863 - val_accuracy: 0.8050 - val_loss: 5.6955
Epoch 2/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7345 - loss: 5.2288 - val_accuracy: 0.8069 - val_loss: 2.7964
Epoch 3/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7551 - loss: 4.4592 - val_accuracy: 0.8147 - val_loss: 3.3116
Epoch 4/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7812 - loss: 4.4992 - val_accuracy: 0.8069 - val_loss: 2.4049
Epoch 5/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7775 - loss: 1.6671 - val_accuracy: 0.7452 - val_loss: 3.2832
Epoch 6/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7925 - loss: 4.3886 - val_accuracy: 0.7992 - val_loss: 2.7484
Epoch 7/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7869 - loss: 3.8077 - val_accuracy: 0.6197 - val_loss: 3.9851
Epoch 8/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7964 - loss: 3.4257 - val_accuracy: 0.7548 - val_loss: 3.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.6028 - loss: 27.7466 - val_accuracy: 0.7280 - val_loss: 7.3408
Epoch 2/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7212 - loss: 3.2197 - val_accuracy: 0.5418 - val_loss: 7.9533
Epoch 3/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7289 - loss: 2.6557 - val_accuracy: 0.4874 - val_loss: 4.5020
Epoch 4/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7110 - loss: 2.5682 - val_accuracy: 0.5021 - val_loss: 5.2768
Epoch 5/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7222 - loss: 2.2477 - val_accuracy: 0.5460 - val_loss: 3.0335
Epoch 6/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7487 - loss: 1.7839 - val_accuracy: 0.4937 - val_loss: 5.4487
Epoch 7/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7342 - loss: 2.0296 - val_accuracy: 0.5126 - val_loss: 3.4014
Epoch 8/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7295 - loss: 1.9641 - val_accuracy: 0.5314 - val_loss: 5.

In [10]:
# Configuration B - 1 Hidden Layer
import keras
import tensorflow as tf
import random
import numpy as np
from keras.models import Sequential
from keras.layers import Masking, Dense, LSTM, TimeDistributed, Dropout, Normalization
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, cohen_kappa_score

# Set a seed value
seed_value= 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

auc_scores = []
kappa_scores = []

for fold in np.unique(fold_nonrecurrent):
    training = np.argwhere(fold_nonrecurrent != fold).ravel()
    testing = np.argwhere(fold_nonrecurrent == fold).ravel()

    X_train, X_test = X_nonrecurrent[training], X_nonrecurrent[testing]
    y_train, y_test = y_nonrecurrent[training], y_nonrecurrent[testing]

    # Define the model
    keras.backend.clear_session()
    model = Sequential([
    Dense(128, activation='relu', input_shape=(92,)),
    Dense(4, activation='softmax')  # Remove the 64-node layer
])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Define early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model with the new training and validation sets
    history = model.fit(X_train, y_train,
                        epochs=100,
                        validation_split=0.2,
                        verbose=1, # setting this to 0 reduces the amount printed
                        callbacks=[early_stopping])

     # Evaluate the model
    y_pred = model.predict(X_test)

    # AUC
    auc_score = auc(y_test, y_pred)

    # Kappa Score
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)
    kappa_score = cohen_kappa_score(y_true_classes, y_pred_classes)

    auc_scores.append(auc_score)
    kappa_scores.append(kappa_score)

# Calculate the average AUC and Kappa scores across all folds
average_auc = np.mean(auc_scores)
average_kappa = np.mean(kappa_scores)

print(f"Average AUC: {average_auc}")
print(f"Average Kappa: {average_kappa}")


Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7128 - loss: 20.0671 - val_accuracy: 0.6290 - val_loss: 8.0004
Epoch 2/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7527 - loss: 3.3842 - val_accuracy: 0.7379 - val_loss: 5.6998
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7533 - loss: 3.2693 - val_accuracy: 0.7056 - val_loss: 4.9966
Epoch 4/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7624 - loss: 2.8926 - val_accuracy: 0.7198 - val_loss: 3.3671
Epoch 5/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7727 - loss: 1.8512 - val_accuracy: 0.7339 - val_loss: 3.1597
Epoch 6/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7743 - loss: 2.0355 - val_accuracy: 0.6391 - val_loss: 4.8288
Epoch 7/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7645 - loss: 2.0432 - val_accuracy: 0.7641 - val_loss: 4.1906
Epoch 8/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7713 - loss: 2.2962 - val_accuracy: 0.7581 - val_loss: 4.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


62/62 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7082 - loss: 13.0154 - val_accuracy: 0.8020 - val_loss: 4.5278
Epoch 2/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7736 - loss: 4.2760 - val_accuracy: 0.7816 - val_loss: 5.6766
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7767 - loss: 2.5381 - val_accuracy: 0.8347 - val_loss: 4.4435
Epoch 4/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7670 - loss: 4.1995 - val_accuracy: 0.6347 - val_loss: 4.3873
Epoch 5/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7755 - loss: 3.7790 - val_accuracy: 0.8286 - val_loss: 4.1906
Epoch 6/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7830 - loss: 2.9980 - val_accuracy: 0.7878 - val_loss: 3.0110
Epoch 7/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7856 - loss: 2.6999 - val_accuracy: 0.7612 - val_loss: 3.7970
Epoch 8/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7895 - loss: 3.1343 - val_accuracy: 0.6837 - val_loss: 3.

...

64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.7104 - loss: 31.6141 - val_accuracy: 0.5424 - val_loss: 8.4322
Epoch 2/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.7225 - loss: 6.1948 - val_accuracy: 0.6312 - val_loss: 5.3135
Epoch 3/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7294 - loss: 4.5145 - val_accuracy: 0.5582 - val_loss: 5.2471
Epoch 4/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.7390 - loss: 4.1544 - val_accuracy: 0.4951 - val_loss: 5.1849
Epoch 5/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7472 - loss: 3.6974 - val_accuracy: 0.5128 - val_loss: 4.3685
Epoch 6/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7279 - loss: 3.4876 - val_accuracy: 0.4103 - val_loss: 5.3059
Epoch 7/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7507 - loss: 3.2268 - val_accuracy: 0.4813 - val_loss: 5.1731
Epoch 8/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7400 - loss: 2.3193 - val_accuracy: 0.6667 - val_loss: 3

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.6914 - loss: 19.3309 - val_accuracy: 0.8031 - val_loss: 3.5338
Epoch 2/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7650 - loss: 5.2558 - val_accuracy: 0.5560 - val_loss: 8.4120
Epoch 3/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7465 - loss: 5.3957 - val_accuracy: 0.7722 - val_loss: 2.8207
Epoch 4/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7915 - loss: 3.2123 - val_accuracy: 0.5483 - val_loss: 4.0421
Epoch 5/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7694 - loss: 2.7883 - val_accuracy: 0.7375 - val_loss: 3.7383
Epoch 6/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7776 - loss: 2.2442 - val_accuracy: 0.7954 - val_loss: 3.0060
Epoch 7/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7782 - loss: 3.1472 - val_accuracy: 0.7510 - val_loss: 3.2902
Epoch 8/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7729 - loss: 3.0401 - val_accuracy: 0.7625 - val_loss: 4.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.6143 - loss: 16.6521 - val_accuracy: 0.6778 - val_loss: 11.0446
Epoch 2/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7262 - loss: 4.4409 - val_accuracy: 0.5105 - val_loss: 6.5534
Epoch 3/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7214 - loss: 3.1237 - val_accuracy: 0.6360 - val_loss: 7.8256
Epoch 4/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7395 - loss: 2.8205 - val_accuracy: 0.5418 - val_loss: 5.6384
Epoch 5/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7362 - loss: 2.0574 - val_accuracy: 0.5021 - val_loss: 5.9546
Epoch 6/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7317 - loss: 2.4687 - val_accuracy: 0.6548 - val_loss: 7.6813
Epoch 7/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7569 - loss: 2.1898 - val_accuracy: 0.5900 - val_loss: 3.9268
Epoch 8/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7468 - loss: 1.6304 - val_accuracy: 0.7531 - val_loss: 

In [13]:
# Configuration C - 2 Hidden Layers with Tanh Activation
import keras
import tensorflow as tf
import random
import numpy as np
from keras.models import Sequential
from keras.layers import Masking, Dense, LSTM, TimeDistributed, Dropout, Normalization
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, cohen_kappa_score

# Set a seed value
seed_value= 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

auc_scores = []
kappa_scores = []

for fold in np.unique(fold_nonrecurrent):
    training = np.argwhere(fold_nonrecurrent != fold).ravel()
    testing = np.argwhere(fold_nonrecurrent == fold).ravel()

    X_train, X_test = X_nonrecurrent[training], X_nonrecurrent[testing]
    y_train, y_test = y_nonrecurrent[training], y_nonrecurrent[testing]

    # Define the model
    keras.backend.clear_session()
    model = Sequential([  # ← This line needs proper indentation!
        Dense(128, activation='tanh', input_shape=(92,)),
        Dense(64, activation='tanh'),
        Dense(4, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Define early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model with the new training and validation sets
    history = model.fit(X_train, y_train,
                        epochs=100,
                        validation_split=0.2,
                        verbose=1,
                        callbacks=[early_stopping])

     # Evaluate the model
    y_pred = model.predict(X_test)

    # AUC
    auc_score = auc(y_test, y_pred)

    # Kappa Score
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)
    kappa_score = cohen_kappa_score(y_true_classes, y_pred_classes)

    auc_scores.append(auc_score)
    kappa_scores.append(kappa_score)

# Calculate the average AUC and Kappa scores across all folds
average_auc = np.mean(auc_scores)
average_kappa = np.mean(kappa_scores)

print(f"Average AUC: {average_auc}")
print(f"Average Kappa: {average_kappa}")

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7538 - loss: 0.7287 - val_accuracy: 0.8246 - val_loss: 0.6804
Epoch 2/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8324 - loss: 0.5771 - val_accuracy: 0.8185 - val_loss: 0.6548
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8329 - loss: 0.5698 - val_accuracy: 0.8185 - val_loss: 0.6650
Epoch 4/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8350 - loss: 0.5580 - val_accuracy: 0.8266 - val_loss: 0.6706
Epoch 5/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8336 - loss: 0.5569 - val_accuracy: 0.8246 - val_loss: 0.6555
Epoch 6/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8344 - loss: 0.5517 - val_accuracy: 0.8306 - val_loss: 0.6587
Epoch 7/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8382 - loss: 0.5420 - val_accuracy: 0.8286 - val_loss: 0.6536
Epoch 8/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8374 - loss: 0.5365 - val_accuracy: 0.8286 - val_loss: 0.6

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.6931 - loss: 0.9149 - val_accuracy: 0.8224 - val_loss: 0.6694
Epoch 2/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8294 - loss: 0.5523 - val_accuracy: 0.8204 - val_loss: 0.6689
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8326 - loss: 0.5381 - val_accuracy: 0.8204 - val_loss: 0.6744
Epoch 4/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8290 - loss: 0.5348 - val_accuracy: 0.8286 - val_loss: 0.6771
Epoch 5/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8278 - loss: 0.5173 - val_accuracy: 0.8245 - val_loss: 0.6846
Epoch 6/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8270 - loss: 0.5144 - val_accuracy: 0.8204 - val_loss: 0.6793
Epoch 7/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8334 - loss: 0.5113 - val_accuracy: 0.8286 - val_loss: 0.6776
Epoch 8/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8288 - loss: 0.5111 - val_accuracy: 0.8327 - val_loss: 0.6

...

64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.7526 - loss: 0.7468 - val_accuracy: 0.8304 - val_loss: 0.6655
Epoch 2/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8130 - loss: 0.6193 - val_accuracy: 0.8304 - val_loss: 0.6471
Epoch 3/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8170 - loss: 0.6125 - val_accuracy: 0.8304 - val_loss: 0.6428
Epoch 4/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8147 - loss: 0.6006 - val_accuracy: 0.8245 - val_loss: 0.6451
Epoch 5/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8138 - loss: 0.6004 - val_accuracy: 0.8245 - val_loss: 0.6526
Epoch 6/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8132 - loss: 0.5911 - val_accuracy: 0.8284 - val_loss: 0.6596
Epoch 7/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8165 - loss: 0.5812 - val_accuracy: 0.8264 - val_loss: 0.6539
Epoch 8/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8151 - loss: 0.5812 - val_accuracy: 0.8284 - val_loss: 0.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.6858 - loss: 0.8912 - val_accuracy: 0.8533 - val_loss: 0.6202
Epoch 2/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8325 - loss: 0.5673 - val_accuracy: 0.8533 - val_loss: 0.6363
Epoch 3/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8326 - loss: 0.5617 - val_accuracy: 0.8552 - val_loss: 0.6362
Epoch 4/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8350 - loss: 0.5493 - val_accuracy: 0.8552 - val_loss: 0.6436
Epoch 5/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8387 - loss: 0.5396 - val_accuracy: 0.8552 - val_loss: 0.6374
Epoch 6/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8395 - loss: 0.5316 - val_accuracy: 0.8533 - val_loss: 0.6456
Epoch 7/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8358 - loss: 0.5309 - val_accuracy: 0.8475 - val_loss: 0.6601
Epoch 8/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8339 - loss: 0.5253 - val_accuracy: 0.8514 - val_loss: 0.6

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.7328 - loss: 0.8360 - val_accuracy: 0.7845 - val_loss: 0.7894
Epoch 2/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8043 - loss: 0.6162 - val_accuracy: 0.7845 - val_loss: 0.8105
Epoch 3/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8073 - loss: 0.6017 - val_accuracy: 0.7845 - val_loss: 0.8181
Epoch 4/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8070 - loss: 0.5911 - val_accuracy: 0.7845 - val_loss: 0.8148
Epoch 5/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8099 - loss: 0.5892 - val_accuracy: 0.7845 - val_loss: 0.8068
Epoch 6/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8094 - loss: 0.5795 - val_accuracy: 0.7866 - val_loss: 0.8120
Epoch 7/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8107 - loss: 0.5740 - val_accuracy: 0.7866 - val_loss: 0.8197
Epoch 8/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8119 - loss: 0.5716 - val_accuracy: 0.7845 - val_loss: 0.

In [16]:
# Configuration D - 3 Hidden Layers with Mixed Activations
import keras
import tensorflow as tf
import random
import numpy as np
from keras.models import Sequential
from keras.layers import Masking, Dense, LSTM, TimeDistributed, Dropout, Normalization
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, cohen_kappa_score

# Set a seed value
seed_value= 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

auc_scores = []
kappa_scores = []

for fold in np.unique(fold_nonrecurrent):
    training = np.argwhere(fold_nonrecurrent != fold).ravel()
    testing = np.argwhere(fold_nonrecurrent == fold).ravel()

    X_train, X_test = X_nonrecurrent[training], X_nonrecurrent[testing]
    y_train, y_test = y_nonrecurrent[training], y_nonrecurrent[testing]

    # Define the model
    keras.backend.clear_session()  # ← Fixed indentation
    model = Sequential([           # ← Fixed indentation
        Dense(64, activation='relu', input_shape=(92,)),
        Dense(64, activation='tanh'),
        Dense(32, activation='tanh'),
        Dense(4, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Define early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model with the new training and validation sets
    history = model.fit(X_train, y_train,
                        epochs=100,
                        validation_split=0.2,
                        verbose=1,
                        callbacks=[early_stopping])

     # Evaluate the model
    y_pred = model.predict(X_test)

    # AUC
    auc_score = auc(y_test, y_pred)

    # Kappa Score
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)
    kappa_score = cohen_kappa_score(y_true_classes, y_pred_classes)

    auc_scores.append(auc_score)
    kappa_scores.append(kappa_score)

# Calculate the average AUC and Kappa scores across all folds
average_auc = np.mean(auc_scores)
average_kappa = np.mean(kappa_scores)

print(f"Average AUC: {average_auc}")
print(f"Average Kappa: {average_kappa}")

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7493 - loss: 0.7604 - val_accuracy: 0.8306 - val_loss: 0.6506
Epoch 2/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8339 - loss: 0.5753 - val_accuracy: 0.8327 - val_loss: 0.6624
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8344 - loss: 0.5709 - val_accuracy: 0.8327 - val_loss: 0.6443
Epoch 4/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8345 - loss: 0.5650 - val_accuracy: 0.8367 - val_loss: 0.6464
Epoch 5/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8339 - loss: 0.5555 - val_accuracy: 0.8347 - val_loss: 0.6411
Epoch 6/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8344 - loss: 0.5554 - val_accuracy: 0.8327 - val_loss: 0.6479
Epoch 7/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8352 - loss: 0.5542 - val_accuracy: 0.8327 - val_loss: 0.6472
Epoch 8/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8349 - loss: 0.5581 - val_accuracy: 0.8306 - val_loss: 0.6

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


62/62 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.6894 - loss: 0.8584 - val_accuracy: 0.8367 - val_loss: 0.6555
Epoch 2/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8311 - loss: 0.5672 - val_accuracy: 0.8367 - val_loss: 0.6714
Epoch 3/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8285 - loss: 0.5562 - val_accuracy: 0.8367 - val_loss: 0.6609
Epoch 4/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8316 - loss: 0.5535 - val_accuracy: 0.8367 - val_loss: 0.6825
Epoch 5/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8334 - loss: 0.5486 - val_accuracy: 0.8367 - val_loss: 0.6685
Epoch 6/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8319 - loss: 0.5552 - val_accuracy: 0.8367 - val_loss: 0.6705
Epoch 7/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8327 - loss: 0.5480 - val_accuracy: 0.8367 - val_loss: 0.6700
Epoch 8/100
62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8321 - loss: 0.5418 - val_accuracy: 0.8367 - val_loss: 0.6

...

64/64 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.7225 - loss: 0.8087 - val_accuracy: 0.8343 - val_loss: 0.6377
Epoch 2/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8169 - loss: 0.6327 - val_accuracy: 0.8343 - val_loss: 0.6397
Epoch 3/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.8162 - loss: 0.6170 - val_accuracy: 0.8343 - val_loss: 0.6395
Epoch 4/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8166 - loss: 0.6242 - val_accuracy: 0.8343 - val_loss: 0.6312
Epoch 5/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8172 - loss: 0.6203 - val_accuracy: 0.8304 - val_loss: 0.6477
Epoch 6/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8099 - loss: 0.6231 - val_accuracy: 0.8343 - val_loss: 0.6401
Epoch 7/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8167 - loss: 0.6253 - val_accuracy: 0.8264 - val_loss: 0.6399
Epoch 8/100
64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8098 - loss: 0.6184 - val_accuracy: 0.8343 - val_loss: 0.

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


65/65 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - accuracy: 0.7573 - loss: 0.7586 - val_accuracy: 0.8571 - val_loss: 0.5847
Epoch 2/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8319 - loss: 0.5766 - val_accuracy: 0.8571 - val_loss: 0.5984
Epoch 3/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8321 - loss: 0.5696 - val_accuracy: 0.8571 - val_loss: 0.5893
Epoch 4/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8322 - loss: 0.5718 - val_accuracy: 0.8571 - val_loss: 0.6111
Epoch 5/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8322 - loss: 0.5644 - val_accuracy: 0.8571 - val_loss: 0.6104
Epoch 6/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8322 - loss: 0.5537 - val_accuracy: 0.8571 - val_loss: 0.5953
Epoch 7/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8320 - loss: 0.5558 - val_accuracy: 0.8571 - val_loss: 0.5921
Epoch 8/100
65/65 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8318 - loss: 0.5500 - val_accuracy: 0.8571 - val_loss: 0.5

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


60/60 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - accuracy: 0.7679 - loss: 0.7376 - val_accuracy: 0.7845 - val_loss: 0.7892
Epoch 2/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8087 - loss: 0.6295 - val_accuracy: 0.7845 - val_loss: 0.8080
Epoch 3/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8065 - loss: 0.6195 - val_accuracy: 0.7845 - val_loss: 0.7966
Epoch 4/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8080 - loss: 0.6170 - val_accuracy: 0.7845 - val_loss: 0.7980
Epoch 5/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8091 - loss: 0.6202 - val_accuracy: 0.7845 - val_loss: 0.8030
Epoch 6/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8093 - loss: 0.6101 - val_accuracy: 0.7803 - val_loss: 0.7896
Epoch 7/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7983 - loss: 0.6148 - val_accuracy: 0.7845 - val_loss: 0.7953
Epoch 8/100
60/60 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8088 - loss: 0.6057 - val_accuracy: 0.7824 - val_loss: 0.8

## Part 2: Long Short Term Memory (LSTM) Neural Network

The code cell below formats the data for a recurrent model (such as a LSTM Neural Network). Remember that the data needs to be represented in 3 dimensions as opposed to 2 (note the difference in data shape as compared to above). In this cell, padding is also applied to the data and a mask is generated so that the model ignores any padded values (this is for model training efficiency).

The second code cell applies a 5-fold cross-validation on a LSTM Neural Network following the structure suggested by Botelho et al. (2017).

[Botelho, A. F., Baker, R. S., & Heffernan, N. T. (2017, June). Improving Sensor-Free Affect Detection Using Deep Learning. *In Proceedings of the 2017 International Conference on Artificial Intelligence in Education*, 40-51. Springer, Cham.](https://link.springer.com/chapter/10.1007/978-3-319-61425-0_4)


**Please follow the instructions in the ASSISTments assignment for modifying and running the cross-validation code cell.**

In [18]:
sequence_lengths = []
X_recurrent = []
y_recurrent = []
mask_recurrent = []
fold_recurrent = []

def parse_data(df):
    if (df[TARGET_FEATURES].sum(axis=1) == 1).sum() > 0:
        keepers = df[EXPERT_FEATURES[0]].notna()
        df = df[keepers]
        sequence_lengths.append(len(df))
        X_recurrent.append(df[EXPERT_FEATURES].values.reshape(-1, len(EXPERT_FEATURES)))
        y_recurrent.append(df[TARGET_FEATURES].values.reshape(-1, len(TARGET_FEATURES)))
        mask_recurrent.append(df[TARGET_FEATURES].sum(axis=1).values.reshape(-1, 1))
        fold_recurrent.append(df['fold'].iloc[0])

def pad_data(a, max_length):
    pad = np.zeros((max_length - a.shape[0], a.shape[1]))
    return np.concatenate([a, pad])

# Use original_data instead of data
original_data = original_data.sort_values(['user_id', 'clip_sequence', 'row_id']).reset_index()
original_data.groupby(['user_id', 'clip_sequence']).apply(parse_data)

max_length = max(sequence_lengths)
X_recurrent_padded = np.stack([pad_data(i, max_length) for i in X_recurrent])
y_recurrent_padded = np.stack([pad_data(i, max_length) for i in y_recurrent])
mask_recurrent_padded = np.equal(np.stack([pad_data(i, max_length) for i in mask_recurrent]), 1)
fold_recurrent = np.array(fold_recurrent)

print(X_recurrent_padded.shape)
print(y_recurrent_padded.shape)

<ipython-input-18-af611547b237>:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  original_data.groupby(['user_id', 'clip_sequence']).apply(parse_data)


(472, 432, 92)
(472, 432, 4)


In [23]:
import keras
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping
from sklearn.metrics import roc_auc_score, cohen_kappa_score

# Set a seed value
seed_value= 42
np.random.seed(seed_value)
random.seed(seed_value)
tf.random.set_seed(seed_value)

auc_scores = []
kappa_scores = []

for fold in np.unique(fold_recurrent):
    training = np.argwhere(fold_recurrent != fold).ravel()
    testing = np.argwhere(fold_recurrent == fold).ravel()

    X_train, X_test = X_recurrent_padded[training], X_recurrent_padded[testing]
    y_train, y_test = y_recurrent_padded[training], y_recurrent_padded[testing]

    # Flatten the sequences for simpler processing
    # Take the last valid timestep for each sequence
    def get_last_valid_timestep(X, y, mask):
        X_out = []
        y_out = []
        for i in range(len(X)):
            # Find last non-zero timestep
            valid_steps = np.any(X[i] != 0, axis=1)
            if np.any(valid_steps):
                last_valid_idx = np.where(valid_steps)[0][-1]
                X_out.append(X[i, last_valid_idx, :])
                y_out.append(y[i, last_valid_idx, :])
        return np.array(X_out), np.array(y_out)

    X_train_simple, y_train_simple = get_last_valid_timestep(X_train, y_train, None)
    X_test_simple, y_test_simple = get_last_valid_timestep(X_test, y_test, None)

    # Only keep samples with valid labels (sum = 1)
    train_valid = y_train_simple.sum(axis=1) == 1
    test_valid = y_test_simple.sum(axis=1) == 1

    if np.sum(train_valid) == 0 or np.sum(test_valid) == 0:
        print(f"Fold {fold}: No valid samples, skipping")
        continue

    X_train_final = X_train_simple[train_valid]
    y_train_final = y_train_simple[train_valid]
    X_test_final = X_test_simple[test_valid]
    y_test_final = y_test_simple[test_valid]

    print(f"Fold {fold}: Train samples: {len(X_train_final)}, Test samples: {len(X_test_final)}")

    # Define a simple feed-forward model
    keras.backend.clear_session()
    model = Sequential([
        Dense(128, activation='relu', input_shape=(92,)),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(4, activation='softmax')
    ])

    # Compile the model
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Define early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    history = model.fit(X_train_final, y_train_final,
                        epochs=100,
                        validation_split=0.2,
                        verbose=1,
                        callbacks=[early_stopping])

    # Evaluate the model
    y_pred = model.predict(X_test_final)

    # AUC
    auc_score = auc(y_test_final, y_pred)

    # Kappa Score
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test_final, axis=1)
    kappa_score = cohen_kappa_score(y_true_classes, y_pred_classes)

    auc_scores.append(auc_score)
    kappa_scores.append(kappa_score)

    print(f"Fold {fold}: AUC = {auc_score:.4f}, Kappa = {kappa_score:.4f}")

# Calculate the average AUC and Kappa scores across all folds
print(f"\nAverage Test Set AUC: {np.mean(auc_scores):.4f}")
print(f"Average Test Set Kappa: {np.mean(kappa_scores):.4f}")

Fold 0: Train samples: 31, Test samples: 11


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.2500 - loss: 126.3800 - val_accuracy: 0.1429 - val_loss: 73.8450
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 397ms/step - accuracy: 0.4167 - loss: 61.7966 - val_accuracy: 0.0000e+00 - val_loss: 57.5561
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step - accuracy: 0.3333 - loss: 84.4571 - val_accuracy: 0.0000e+00 - val_loss: 42.6023
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - accuracy: 0.2500 - loss: 90.6415 - val_accuracy: 0.0000e+00 - val_loss: 31.8779
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.4167 - loss: 84.8375 - val_accuracy: 0.2857 - val_loss: 21.8983
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - accuracy: 0.4167 - loss: 44.7775 - val_accuracy: 0.2857 - val_loss: 17.0381
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step - accuracy: 0.2917 - loss: 56.3357 - val_accuracy: 0.2857 - val_loss: 12.8404
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.3750 - loss: 74.1946 -

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.2963 - loss: 94.9751 - val_accuracy: 0.2857 - val_loss: 12.8160
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step - accuracy: 0.1852 - loss: 74.4454 - val_accuracy: 0.7143 - val_loss: 11.0116
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 182ms/step - accuracy: 0.4444 - loss: 31.2644 - val_accuracy: 0.7143 - val_loss: 14.1882
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step - accuracy: 0.3333 - loss: 68.7222 - val_accuracy: 0.7143 - val_loss: 17.4080
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.4815 - loss: 38.3631 - val_accuracy: 0.7143 - val_loss: 20.2134
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.3333 - loss: 55.7280 - val_accuracy: 0.7143 - val_loss: 22.5751
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - accuracy: 0.5185 - loss: 49.3299 - val_accuracy: 0.7143 - val_loss: 23.8859
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - accuracy: 0.4074 - loss: 35.8541 - val_accuracy: 0.7143 - va

...

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.1852 - loss: 231.9701 - val_accuracy: 0.0000e+00 - val_loss: 48.6841
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.2222 - loss: 164.4011 - val_accuracy: 0.1429 - val_loss: 24.3854
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.1852 - loss: 126.7923 - val_accuracy: 0.5714 - val_loss: 3.8276
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.2222 - loss: 103.1264 - val_accuracy: 0.8571 - val_loss: 2.0164
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.4074 - loss: 52.6965 - val_accuracy: 0.7143 - val_loss: 3.3190
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step - accuracy: 0.3704 - loss: 57.2468 - val_accuracy: 0.7143 - val_loss: 4.5247
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.4815 - loss: 49.6286 - val_accuracy: 0.7143 - val_loss: 5.6791
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.5556 - loss: 80.6150 - val_accuracy: 0.7143 - 

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step
Fold 3: AUC = 0.4188, Kappa = 0.0000
Fold 4: Train samples: 34, Test samples: 8
Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.1481 - loss: 144.3200 - val_accuracy: 0.0000e+00 - val_loss: 91.1467
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 419ms/step - accuracy: 0.0741 - loss: 105.9552 - val_accuracy: 0.0000e+00 - val_loss: 75.9231
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.1852 - loss: 113.7309 - val_accuracy: 0.0000e+00 - val_loss: 59.5858
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - accuracy: 0.3704 - loss: 52.9074 - val_accuracy: 0.0000e+00 - val_loss: 45.5124
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - accuracy: 0.2222 - loss: 72.0640 - val_accuracy: 0.1429 - val_loss: 37.8763
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 133ms/step - accuracy: 0.2593 - loss: 45.4384 - val_accuracy: 0.4286 - val_loss: 40.9441
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step - accuracy: 0.3333 - loss: 52.9485 - val_accuracy: 0.5714 - val_loss: 46.7584
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step - accuracy: 0.4815 - loss: 56.2747 - val_ac